In [1]:
import os
import sys
import requests
from utils.experiment import Experiment, execute_command_on_server_and_clients
from execo_g5k import oarsub, oardel, OarSubmission, get_current_oar_jobs, get_oar_job_nodes, get_oar_job_info, Deployment, deploy

In [2]:
jobname="fl-measure"
nodecount=11
walltime="13:50:00"
resources_selection="-t exotic -p estats -t night"
site="toulouse"
force_redeploy=False
environment_dsc_file='../images/fl_jetson_image_complet.yaml'
storage_group="energyfl"

# Reserve a job and deploy the chosen environment

In [3]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
ifdeploy=True

In [4]:
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            ifdeploy=False
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname,
                            queue='default')
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 450875


In [4]:
jobid = 450875
nodes = get_oar_job_nodes(jobid, site)
nodes.sort(key=lambda n: n.address)
nodes

[Host('estats-10.toulouse.grid5000.fr'),
 Host('estats-11.toulouse.grid5000.fr'),
 Host('estats-12.toulouse.grid5000.fr'),
 Host('estats-2.toulouse.grid5000.fr'),
 Host('estats-3.toulouse.grid5000.fr'),
 Host('estats-4.toulouse.grid5000.fr'),
 Host('estats-5.toulouse.grid5000.fr'),
 Host('estats-6.toulouse.grid5000.fr'),
 Host('estats-7.toulouse.grid5000.fr'),
 Host('estats-8.toulouse.grid5000.fr'),
 Host('estats-9.toulouse.grid5000.fr')]

In [5]:
if ifdeploy:
    deployment = Deployment(hosts=nodes, env_file=os.path.abspath(environment_dsc_file))
    print(os.path.abspath(environment_dsc_file))
    deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                                stdout_handlers=[sys.stdout],
                                stderr_handlers=[sys.stderr])
    print("Deployement status:\n* ok: {}\n* failed: {}".format(deploy_ok, deploy_failed))

/home/tunguyen/jetson-test/images/fl_jetson_image_complet.yaml
Deployement status:
* ok: {'estats-5.toulouse.grid5000.fr', 'estats-7.toulouse.grid5000.fr', 'estats-3.toulouse.grid5000.fr', 'estats-12.toulouse.grid5000.fr', 'estats-4.toulouse.grid5000.fr', 'estats-10.toulouse.grid5000.fr', 'estats-11.toulouse.grid5000.fr', 'estats-6.toulouse.grid5000.fr', 'estats-2.toulouse.grid5000.fr', 'estats-9.toulouse.grid5000.fr', 'estats-8.toulouse.grid5000.fr'}
* failed: set()


toulouse: Warning: Permanently added 'toulouse.grid5000.fr,172.16.127.110' (ECDSA) to the list of known hosts.toulouse: 

toulouse: Deployment #D-cf8513c9-8c45-40f0-8af7-1218e0c8a815 startedtoulouse: toulouse: Grab the key file /home/tunguyen/.ssh/authorized_keystoulouse: toulouse: Grab the tarball file /home/tunguyen/public/ubuntul4t35-pytorch-estats.tar.zsttoulouse: toulouse: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgztoulouse: toulouse: Using classical reboot instead of kexec (SetDeploymentMiniOS) because the last deployment is not trustedtoulouse: toulouse: Launching a deployment of ubuntul4t35-pytorch-estats-datasets:tunguyen:aarch64:2024030113 on estats-[2-12].toulouse.grid5000.frtoulouse: toulouse: Performing a Deploy[SetDeploymentMiniOSUntrusted] steptoulouse: toulouse:   switch_pxetoulouse: toulouse: [dbg] -------------------------toulouse: toulouse: [dbg] NODE: estats-2.toulouse.grid5000.frtoulouse: toulouse: [dbg] -------------------------toulouse: toulouse: [dbg] COMMAND: KADEPLOY_BOOTLOADER_NO_GRUB_FROM_DEST="1" KADEPLOY_BOOTLOADER_NO_GRUB_MKCONFIG_FROM_DEST=

toulouse: Connection to toulouse.grid5000.fr closed.toulouse: 

# Allow access for the node to the nfs storage group and mount the storage to a folder

In [6]:
#!curl -X POST 'https://api.grid5000.fr/stable/sites/toulouse/storage/storage1/energyfl/access' -H "Content-Type: application/json" -d '{"termination" : {"job":450847, "site": "toulouse"}}'
# or
if ifdeploy:
    url = f"https://api.grid5000.fr/stable/sites/{site}/storage/storage1/{storage_group}/access"
    requests.post(url, headers={"Content-Type": "application/json"}, json={"termination": {"job": jobid, "site": site}}, verify=False)
    cmd = f"mkdir -p /root/{storage_group} ; mount storage1.toulouse.grid5000.fr:/export/group/{storage_group} /root/{storage_group}/"
    _ = execute_command_on_server_and_clients(nodes, cmd, background=False) 

/home/tunguyen/venv-fl/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.grid5000.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Successfully executed on Host('estats-10.toulouse.grid5000.fr') command 'mkdir -p /root/energyfl ; mount storage1.toulouse.grid5000.fr:/export/group/energyfl /root/energyfl/'
Successfully executed on Host('estats-11.toulouse.grid5000.fr') command 'mkdir -p /root/energyfl ; mount storage1.toulouse.grid5000.fr:/export/group/energyfl /root/energyfl/'
Successfully executed on Host('estats-12.toulouse.grid5000.fr') command 'mkdir -p /root/energyfl ; mount storage1.toulouse.grid5000.fr:/export/group/energyfl /root/energyfl/'
Successfully executed on Host('estats-2.toulouse.grid5000.fr') command 'mkdir -p /root/energyfl ; mount storage1.toulouse.grid5000.fr:/export/group/energyfl /root/energyfl/'
Successfully executed on Host('estats-3.toulouse.grid5000.fr') command 'mkdir -p /root/energyfl ; mount storage1.toulouse.grid5000.fr:/export/group/energyfl /root/energyfl/'
Successfully executed on Host('estats-4.toulouse.grid5000.fr') command 'mkdir -p /root/energyfl ; mount storage1.toulouse.grid5

# Define parameters for FL experiment and run it

In [ ]:
params = {
    "params.num_rounds":[10],
    "data.num_clients": [10],
    "data.alpha": [10], #[1,2,5,10],
    "data.partition":["label_skew"],
    "client.lr" : [1e-2],#,1e-2],
    "client.local_epochs": [1],
    "client.decay_rate": [1],
    "client.decay_steps": [10],
    "neuralnet":["MobileNetV3Small"],
    "strategy": ["fedavg"],
    "optimizer": ["SGD"],
}

repository_dir = "/home/tunguyen/jetson-test"

to_remove = ["client.cid","client.dry_run","data.partition_dir","params.num_classes","tmp_result_folder","exp_datetime"]

Exps = Experiment(
    params=params,
    nodes=nodes,
    repository_dir=repository_dir,
    sleep=30,
    key_to_remove=to_remove)
#Exps.frontend_dry_run()
Exps.run()

In [ ]:
Exps.kill_all()

Results will be available at `f"/srv/storage/{group_storage}@storage1.toulouse.grid5000.fr"`

# And don't forget to kill the job when you're done with the experiments

In [ ]:
oardel([(jobid,site)])